In [ ]:
putStrLn "OK"

## Reader Monad

In [ ]:
newtype R env a = R {runR :: env ->  a}

instance Functor (R env) where
  fmap f (R e) = R(f . e)

instance Applicative (R env) where
  pure x = R (const x)
  R f <*> R ea = R (\e -> (f e) (ea e))

instance Monad (R env) where
  return  = pure
  rx >>= f = R(\e -> let a = runR rx e
                     in runR (f a) e)

ask :: R env env
ask = R(id)

asks :: (e -> a) -> R e a
asks f = ask >>= (pure . f) 


#### Test

In [ ]:
env = [1,3,3,45]
sm :: R [Int] Int
sm = do
        f <- asks (!! 0)
        s <- asks (!! 1)
        return (f + s)

In [ ]:
runR sm env

In [ ]:
env = [1,3,3,45]
sm :: R [Int] Int
sm = asks (!! 0) >>= (\f -> asks (!! 1) >>= (\s -> return (f + s)))

In [ ]:
runR sm [1,7,8]

In [ ]:
(runR . R) (+1) 2

## Writer Monad

In [3]:
newtype Writer w a = Writer { runW :: (a,w) }  deriving Show

instance (Monoid w) => Functor (Writer w) where
  fmap f (Writer (x,w)) = Writer (f x, w)

instance (Monoid  w) => Applicative (Writer w) where
  pure x = Writer (x,mempty)
  (Writer (f,w)) <*> (Writer (x,w')) = Writer (f x, w `mappend` w')


instance (Monoid w) => Monad (Writer w) where  
    return x = Writer (x, mempty)  
    (Writer (x,v)) >>= f = let (Writer (y, v')) = f x in Writer (y, v `mappend` v')  

tell :: (Monoid w) => w -> Writer w ()
tell x = Writer((),x)
set :: (Monoid w) => m -> Writer w m
set x = Writer (x,mempty) 

In [5]:
newtype LogEntry = LogEntry { msg::String }
  deriving (Eq, Show)

calc :: Writer [LogEntry] Integer
calc = do
  output "start"
  let x = sum [1..10000000]
  output (show x)
  output "done"
  return x

output :: String -> Writer [LogEntry] ()
output x = tell [LogEntry x]

test = mapM_ print $ runW calc

In [6]:
test

[LogEntry {msg = "start"},LogEntry {msg = "50000005000000"},LogEntry {msg = "done"}]

In [7]:
runW calc

(50000005000000,[LogEntry {msg = "start"},LogEntry {msg = "50000005000000"},LogEntry {msg = "done"}])

In [12]:
mapM print (1, Just 4)

Just 4
(1,())

#### Test

In [ ]:
log = Writer (0,[])

In [ ]:
log >>= (\x -> Writer (x+1,[x]) >>= (\y -> return y))

In [ ]:
inc = \x -> Writer (x+1,[x])
        

In [ ]:
logs = do
        Writer(0,[])
        Writer(1,[0])
        Writer(2,[1])

In [ ]:
runW logs

In [ ]:
logger = Writer (0,"")
ll l = do
      x <- l
      return x

In [ ]:
ll Writer(1,"ciao")

In [ ]:
tell :: (Monoid w) => w -> Writer w ()
tell x = Writer((),x)
set :: (Monoid w) => m -> Writer w m
set x = Writer (x,mempty) 

In [ ]:
runW (tell [1])

In [ ]:
l = do
      logger
      tell "ciao"
      tell " hello"
      set 4

In [ ]:
runW l

## State

In [9]:
newtype ST st a = S {runS :: st -> (a,st)}

instance Functor (ST a)  where
  fmap f st = S(\s -> let (x,s') = runS st s
                      in (f x, s'))

instance Applicative (ST a) where
  pure x = S (\s -> (x,s))
  fs <*> st = S(\s -> let (f,s') = runS fs s
                          (x,s'') = runS st s'
                      in (f x, s''))  

instance Monad (ST a) where
  return x = S(\st -> (x,st))
  m >>= f = S(\s -> let (x',s') = runS m s
                    in runS (f x') s')
                    -- in (x'',s''))

get :: ST s s
get = S(\s -> (s,s))

put :: s -> ST s ()
put x = S(const ((),x))

modify :: (s -> s) -> ST s ()
modify f = S (\s' -> ((), f s')



Line 8: Use tuple-section
Found:
\ s -> (x, s)
Why not:
(x,)Line 14: Use tuple-section
Found:
\ st -> (x, st)
Why not:
(x,)

In [10]:
push :: Int -> ST [Int] Int
push x = S (\s -> (x,x:s))

pop :: ST [Int] Int
pop = S(\(x:xs) -> (x,xs))



In [15]:
go = do 
      push 3
      push 6
      pop

In [14]:
runS go []

(6,[3])

## Monad Transformer

In [45]:
import Control.Monad
import Control.Monad.TR
join (Just (Just 4))
(flip (+)) 1 3
  

Line 3: Redundant bracket
Found:
(flip (+)) 1
Why not:
flip (+) 1Line 3: Redundant flip
Found:
(flip (+)) 1 3
Why not:
(+) 3 1

Just 4

4

In [21]:
do
  y <- Just 4
  (1,Just y)

: 